In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

- now onto motion planning
- so far, you've learned a lot about perception, like how to perceive the world, but while the car brains will taking vision in, eventually you have to also do something with that - you have to steer, and you have to break, and give gas
- motion planning is a step in that direction and it comes in multiple segments
  - it starts out with a predictive task where the motion planner tries to understand what are other vehicles about to do in the future
  - then the vehicle will pick a behavior--kind of simply speaking, whether to go left or right
  - eventually, it generates a maneuver trajectory as to where to go
    - the trajectory is not just a curve in space in X and Y--it also has a time dimension because we have to set the speed of the vehicle at the same time

# Introduction to Path Planning

- path planning is how the vehicle generates safe drivable trajectories to get where we want it to go
- we'll use data from computer vision and sensor fusion in order to understand the environment around us
- we'll also use data from localization to understand precisely where we are in that environment
- the path planning block uses all of that data to decide which maneuver to take next
- then it constructs a trajectory for the controller to execute


- in the first lesson you'll learn some of the foundational search algorithms used in discrete path planning
- then you will learn about prediction which is where we use the data from sensor fusion to generate predictions about what all the other objects around us are likely to do
- then you'll study about behavior planning--this is where we decide at a high level what the car shall do in the next 10 seconds or so
- finally you will learn about trajectory generation which is where we create smooth, drivable, and collision-free trajectories for the motion controller to follow


- in the project at the end of this module you'll implement a path planner for a vehicle driving on a highway
- your vehicle will be able to avoid collisions, maintain safe distances between other traffic, and even pass other vehicles to get where it's going faster

# About this Lesson

- this lesson covers discrete path planning and in the last lesson you will learn about continuous path planning
- even though the real world is continuous, there are many situations where discretizing the world makes it easier and computationally faster to solve path planning problems


- in addition to the practical benefits of these algorithms, it's also conceptually useful to learn about them because they deal with some of the same concepts that we will keep coming back to in this lesson
- those concepts include:
  - cost functions and how to include human insights (like "it's easier to make right turns than left turns") into our planning algorithms
  - optimality and the tradeoffs associated with finding the best solution vs finding a solution that is good enough
  - Online vs Offline algorithms and how we can avoid complex computation on the road by precomputing best paths whenever possible

# Motion Planning

- the fundamental problem in motion planning is that a robot might live in a world like this, and it might want to find its way to a goal like this, and has to devise a plan to get there

<img src="resources/robot_simple_world_goal.png"/>

- this same problem occurs for a self driving car that might live in a city near a highway on a network of streets
- it has to find its way around and navigate to its target location

<img src="resources/robot_simple_world_goal_highway.png"/>

- if we zoom in and look at this intersection, we have also planning problems here
  - imagine a car coming from here (start) that wishes to go over here (goal)
  - to take a left turn on this intersection over here, this car would have to turn right first, engage in a lane shift and then take the left turn to the goal location
  - now, a lane shift over here is a risky proposition--if there's a bit truck parked over here, the space might be insufficient to carry out the lane shift
  - an alternative plan might be to go straight over here, take the detour around the block, and then go straight to the target location

<img src="resources/robot_simple_world_goal_crossroads.png"/>

- the process of finding a path from a start location to a goal location is called *planning*
- for robots, it's often called *robot motion planning*

- in this lesson I'm going to talk about discrete methods for planning in which the world chopped into small bins
- what's the planning problem?
  - we're given:
    - a map of the world
    - a starting location
    - a goal location.
    - usually, we're given some sort of a cost function
      - the simplest way to think of cost is just the time it takes to drive a certain route
  - the goal is find the minimum cost path

# Compute Cost

<img src="resources/compute_cost_quiz.png"/>

- suppose we live in a discrete world like this, and this is a world we'll be programming
- let's for simplicity assume that the world is divided into little grid cells
- our initial location is over here (blue) facing north or up
  - this is the vehicle and the little arrow over here indicates where it's facing
  - I'll call this "Start"
- we wish to get to this area over here (red), presumably facing to the left side


- **Q:** Let's assume at each time step I can either move forward or I can turn the vehicle--I'm going to call these *actions*. Each of those costs me exactly 1 unit of cost. What's the total cost I have to endure to move from start to goal?
- **A:** It's $7$ and not $6$. The reason why it is $7$ is it takes $6$ steps to go on the shortest path to the goal, but in this cell over here I have to turn to the left side. That turn also costs me a unit of $1$. That's in total $6$ straight motions and 1 turn, gives me $7$.


- let me now change the action model into a different model
- we have 3 actions:
  - the first is I can go just forward
  - the second one is I can turn left and then go forward
  - the third one is I can turn right and then go forward
- let's say for the time being the cost of each of those is $1$.


- **Q:** What is now the total cost of the optimal path to the goal?
- **A:** The answer is $6$, as opposed to $7$ when we counted the turning separately.

# Optimal Path

- the reason why I change the actions this way is for my next quiz
- suppose we punish left turns
  - why would we do this?
    - well, in real traffic, left turns are harder to do than the right turns--often you have to wait for oncoming traffic
- let's say in our planning, left turns are more expensive
  - in fact, I should mention that parcel delivery services that plan for optimal routes of trucks like FedEx and UPS in the States, they actually plan routes that try to avoid left turns during rush hours, because it just takes much longer to do left turns
    - if they can go right turns, they prefer those


- **Q:** Let's now say that forward motion will cost you $1$, a left turn costs you $10$, and a right turn costs you $1$. Now, what is the optimal path, and specifically what's the cost of the optimal path?
- **A:** This was a tricky question. It's $15$ it turns out. The path stays the same as the previous quiz, but here instead of $1$ for the turn we add $10$.


- **Q:** So for this final version, let me punish left turns even more. Now they cost us $20$. What is now the total cost to get to the goal? Enter the cost of the optimal (lowest cost) path.
- **A:** The answer is $16$. We take the route around the block--the loop over here. With a cost of left turns this high, the left turn over here becomes prohibitively expensive, and we'd rather take a detour on the right side and do 3 right turns as opposed to 1 left turn.

# Maze

<img src="resources/maze_quiz.png"/>

- let me look at a different maze, and this is not a car example anymore, but it's close to what we're going to program
- suppose we start over here, and our goal is to go into this corner over here--there are multiple blocked cells along the way
- in this case, we have a robot that can go up, down, left, or right


- **Q:** How many steps does it take for the robot that starts over here to reach the goal position?
- **A:** The answer is $7$.

<img src="resources/maze_quiz_2.png"/>

- let's start with a little grid world of size $6 \times 5$ where our start location is in the top left corner, our goal in the bottom right corner and I block off a few cells so there is still a safe path to the goal
- this could be a search through a city graph, through a parking lot, or through a maze of streets for a mobile robot
- just for simplicity, in this example let's assume the robot is given $4$ actions: it can go up, down, left, or right
- also for simplicity, let's assume every action succeeds with absolute certainty
  - we don't model uncertainty in this example
- the path planning or search problem is to find the shortest sequence of actions that leads our robot from the start state to the goal state


- **Q:** Just to check, tell me how many you think these are. How many action are required to go from start to goal?
- **A:** The answer is $11$.

# First Search Program

- the big question now is, can we write a program that finds the shortest path from start to goal?
- to do so, let's give the grid cell names--we have six columns named from $0-5$ and five rows from $0-4$ 
- the basic idea I'll pursue is that I keep the list of nodes that I wish to investigate further, or as we call it in search, *expand*
  - let's call this list open


- so at the beginning we have one state on this list-it's $[0,0]$, my initial state and just to make sure I never pick this state again--I don't want any cycles in my path, let me just check mark the state with little red check
- I now can test whether the state is my final goal state--obviously, it's not--I'm not done with planning yet
- so what I'll do next, I'll expand this state--I'll take it off my open list and look at all the successors of which there are two over here, $[1,0]$ and $[0,1]$--those two are now expanded so I check them
- one last thing I maintain for each of these states on the open list, how many expands that it took to get there--that's called my *g-value*
  - when I'm done with planning, this will be the length of the optimal path
- let's now go further and expand one of the two--we always expand the one with the smallest g-value
- but these are equivalent--they both have a g-value of one, so it doesn't make a difference
- so let me expand the first one--this one has three neighbors, $[0,0]$, $[1,1]$ and $[2,0]$
- because $[0,0]$ is already closed with a checkmark, we don't consider it anymore, which gives me $[2,0]$ and $[1,1]$ both now with a g-value of two and we check those over here
- I now pick the node on the open list with the smallest g-value, which happens to be this one over here--there's really no choice, it's the node over here
- this has two neighbors, $[0,0]$ and $[1,1]$ but both I've already checked so therefore, there is no expansion that takes place
- I only expand if I find an unchecked node


- so a new open list are these two nodes over here and I recurse and what's going to happen is, my nodes will expand gradually into the free space until I eventually hit the goal node and without proof, the g-value when I hit the goal node will be exactly the number of steps it takes to go from the start state to the goal node
- the secret here for that to be the case lies in the fact that I always expand the node with the smallest g-value

In [2]:
# ----------
# User Instructions:
#
# Define a function, search() that returns a list
# in the form of [optimal path length, row, col]. For
# the grid shown below, your function should output
# [11, 4, 5].
#
# If there is no valid path from the start point
# to the goal, your function should return the string
# 'fail'
# ----------

# Grid format:
#   0 = Navigable space
#   1 = Occupied space


grid = [
    [0, 0, 1, 0, 0, 0],
    [0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0],
    [0, 0, 1, 1, 1, 0],
    [0, 0, 0, 0, 1, 0],
]
init = [0, 0]
goal = [len(grid) - 1, len(grid[0]) - 1]
cost = 1

delta = [[-1, 0], [0, -1], [1, 0], [0, 1]]  # go up  # go left  # go down  # go right

delta_name = ["^", "<", "v", ">"]


def search(grid, init, goal, cost):
    # ----------------------------------------
    # insert code here
    # ----------------------------------------

    # list of cells once they're expanded so we don't expand them again
    # assigns an array of the same size as the field grid
    # 0 meaning it's still open, 1 meaning it's being closed
    closed = [[0 for row in range(len(grid[0]))] for col in range(len(grid))]
    # initialize the starting location as checked
    closed[init[0]][init[1]] = 1

    # assign the coordinates to x, y, and a g-value of 0
    x = init[0]
    y = init[1]
    g = 0

    # initial open list is going to be just 1 element of my initial coordinates and the g value of 0
    open = [[g, x, y]]

    # set this flag true if goal is found
    found = False
    # set this flag true if goal is not found and there are no further options to expand
    giveup = False

    # algorithm
    while not found and not giveup:
        # check if we still have elements on the open list
        if len(open) == 0:
            resign = True
        #             print ("fail")
        else:
            # remove node from list i.e. remove the element with the smallest g-value
            open.sort()
            open.reverse()
            next = open.pop()
            x = next[1]
            y = next[2]
            g = next[0]

            # check if we are done
            if x == goal[0] and y == goal[1]:
                found = True
                print(next)
            else:
                # expand winning element and add to new open list
                for i in range(len(delta)):
                    x2 = x + delta[i][0]
                    y2 = y + delta[i][1]
                    if x2 >= 0 and x2 < len(grid) and y2 >= 0 and y2 < len(grid[0]):
                        if closed[x2][y2] == 0 and grid[x2][y2] == 0:
                            g2 = g + cost
                            open.append([g2, x2, y2])
                            closed[x2][y2] = 1

    return [g, x, y]


search(grid, init, goal, cost)

[11, 4, 5]


[11, 4, 5]

<IPython.core.display.Javascript object>

# Expansion Grid

- in the next programming quiz, I would like you to print out a table called *expand*, which does not exist right now
- expand is a table of the same size as grid that maintains at what step each node was expanded
- in this table, every node that has never been expanded, including all the obstacle nodes should have the value of -1
- when a node is expanded, it should get a unique number that is incremented from expansion to expansion and counts from 0, in this case, all the way to 22 for reaching the goal stated
- to give you a second example of how the quotes should work, let me block off the goal by adding 1 over here so there's an entire items that block the left side from the right side
- now the switch fails, and in the expansion list you find that all nodes on the right side have never been expanded

In [3]:
# -----------
# User Instructions:
#
# Modify the function search so that it returns
# a table of values called expand. This table
# will keep track of which step each node was
# expanded.
#
# Make sure that the initial cell in the grid
# you return has the value 0.
# ----------

grid = [
    [0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 1, 0],
    [0, 0, 1, 0, 1, 0],
    [0, 0, 1, 0, 1, 0],
]
init = [0, 0]
goal = [len(grid) - 1, len(grid[0]) - 1]
cost = 1

delta = [[-1, 0], [0, -1], [1, 0], [0, 1]]  # go up  # go left  # go down  # go right

delta_name = ["^", "<", "v", ">"]


def search(grid, init, goal, cost):
    # ----------------------------------------
    # insert code here
    # ----------------------------------------

    # list of cells once they're expanded so we don't expand them again
    # assigns an array of the same size as the field grid
    # 0 meaning it's still open, 1 meaning it's being closed
    closed = [[0 for row in range(len(grid[0]))] for col in range(len(grid))]
    # initialize the starting location as checked
    closed[init[0]][init[1]] = 1

    # make an expand table of exactly the same size as closed but initialized it with -1.
    expand = [[-1 for row in range(len(grid[0]))] for col in range(len(grid))]

    # assign the coordinates to x, y, and a g-value of 0
    x = init[0]
    y = init[1]
    g = 0

    # initial open list is going to be just 1 element of my initial coordinates and the g value of 0
    open = [[g, x, y]]

    # set this flag true if goal is found
    found = False
    # set this flag true if goal is not found and there are no further options to expand
    giveup = False
    # counts the expansion
    count = 0

    # algorithm
    while not found and not giveup:
        # check if we still have elements on the open list
        if len(open) == 0:
            resign = True
        #             print ("fail")
        else:
            # remove node from list i.e. remove the element with the smallest g-value
            open.sort()
            open.reverse()
            next = open.pop()
            x = next[1]
            y = next[2]
            g = next[0]
            expand[x][y] = count
            count += 1

            # check if we are done
            if x == goal[0] and y == goal[1]:
                found = True
                print(next)
            else:
                # expand winning element and add to new open list
                for i in range(len(delta)):
                    x2 = x + delta[i][0]
                    y2 = y + delta[i][1]
                    if x2 >= 0 and x2 < len(grid) and y2 >= 0 and y2 < len(grid[0]):
                        if closed[x2][y2] == 0 and grid[x2][y2] == 0:
                            g2 = g + cost
                            open.append([g2, x2, y2])
                            closed[x2][y2] = 1

    return expand


search(grid, init, goal, cost)

[9, 4, 5]


[[0, 1, -1, 11, 15, 18],
 [2, 3, 5, 8, 12, 16],
 [4, 6, -1, 13, -1, 19],
 [7, 9, -1, 17, -1, 21],
 [10, 14, -1, 20, -1, 22]]

<IPython.core.display.Javascript object>

# Print Path

- I would like you to augment this code to print something entirely different, which is the final solution
- this is nothing to do with expand, and you have to implement a new delta structure for this similar to expand
- print the optimal path with arrows and in the end we find a star, which indicates the location of the goal

In [4]:
# -----------
# User Instructions:
#
# Modify the the search function so that it returns
# a shortest path as follows:
#
# [['>', 'v', ' ', ' ', ' ', ' '],
#  [' ', '>', '>', '>', '>', 'v'],
#  [' ', ' ', ' ', ' ', ' ', 'v'],
#  [' ', ' ', ' ', ' ', ' ', 'v'],
#  [' ', ' ', ' ', ' ', ' ', '*']]
#
# Where '>', '<', '^', and 'v' refer to right, left,
# up, and down motions. Note that the 'v' should be
# lowercase. '*' should mark the goal cell.
#
# You may assume that all test cases for this function
# will have a path from init to goal.
# ----------

grid = [
    [0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 1, 0],
    [0, 0, 1, 0, 1, 0],
    [0, 0, 1, 0, 1, 0],
]
init = [0, 0]
goal = [len(grid) - 1, len(grid[0]) - 1]
cost = 1

delta = [[-1, 0], [0, -1], [1, 0], [0, 1]]  # go up  # go left  # go down  # go right

delta_name = ["^", "<", "v", ">"]


def search(grid, init, goal, cost):
    # ----------------------------------------
    # insert code here
    # ----------------------------------------

    # list of cells once they're expanded so we don't expand them again
    # assigns an array of the same size as the field grid
    # 0 meaning it's still open, 1 meaning it's being closed
    closed = [[0 for row in range(len(grid[0]))] for col in range(len(grid))]
    # initialize the starting location as checked
    closed[init[0]][init[1]] = 1

    # make an expand table of exactly the same size as closed but initialized it with -1.
    expand = [[-1 for row in range(len(grid[0]))] for col in range(len(grid))]

    # make a table to memorize for each cell what action it took to get there
    action = [[-1 for row in range(len(grid[0]))] for col in range(len(grid))]

    # assign the coordinates to x, y, and a g-value of 0
    x = init[0]
    y = init[1]
    g = 0

    # initial open list is going to be just 1 element of my initial coordinates and the g value of 0
    open = [[g, x, y]]

    # set this flag true if goal is found
    found = False
    # set this flag true if goal is not found and there are no further options to expand
    giveup = False
    # counts the expansion
    count = 0

    # algorithm
    while not found and not giveup:
        # check if we still have elements on the open list
        if len(open) == 0:
            resign = True
        #             print ("fail")
        else:
            # remove node from list i.e. remove the element with the smallest g-value
            open.sort()
            open.reverse()
            next = open.pop()
            x = next[1]
            y = next[2]
            g = next[0]
            expand[x][y] = count
            count += 1

            # check if we are done
            if x == goal[0] and y == goal[1]:
                found = True
                print(next)
            else:
                # expand winning element and add to new open list
                for i in range(len(delta)):
                    x2 = x + delta[i][0]
                    y2 = y + delta[i][1]
                    if x2 >= 0 and x2 < len(grid) and y2 >= 0 and y2 < len(grid[0]):
                        if closed[x2][y2] == 0 and grid[x2][y2] == 0:
                            g2 = g + cost
                            open.append([g2, x2, y2])
                            closed[x2][y2] = 1
                            action[x2][y2] = i

    # resulting table output
    policy = [[" " for row in range(len(grid[0]))] for col in range(len(grid))]
    x = goal[0]
    y = goal[1]
    policy[x][y] = "*"
    while x != init[0] or y != init[1]:
        x2 = x - delta[action[x][y]][0]
        y2 = y - delta[action[x][y]][1]
        policy[x2][y2] = delta_name[action[x][y]]
        x = x2
        y = y2

    return policy


search(grid, init, goal, cost)

[9, 4, 5]


[['>', 'v', ' ', ' ', ' ', ' '],
 [' ', '>', '>', '>', '>', 'v'],
 [' ', ' ', ' ', ' ', ' ', 'v'],
 [' ', ' ', ' ', ' ', ' ', 'v'],
 [' ', ' ', ' ', ' ', ' ', '*']]

<IPython.core.display.Javascript object>

- I go from the goal backwards
- I iterate from the goal location, x and y, now in backwards order all the way to the start--do this as long as x and y haven't become my initial location yet
- I apply the inverse action--so I find the originating state by taking my current state and subtracting the action exactly the same way I added it before using my action field as finding out what action was actually being used
- in doing so the first time I do this, x and y was the goal state, and x2, y2 become the state before
- I happen to know in the goal state that the action was a down action
- I then mark the policy field for the originating state to be the special symbol associated with this specific action over here
- then I recourse--I set x and y to the state x2, y2, and I then go a step further--in doing so I will reverse the path step by step, print the associated action, and get exactly this state over here

# A*

- now I want to come with you to the absolute meat of this class, which is called *A-star*
- A* is a variant of the search algorithm that's more efficient than expanding every node
- it is one of the most powerful search algorithms that they use for the present day to drive self-driving cars through unstructured environments
- if you've gotten so far, and you understand the mechanism for searching by gradually expanding nodes in the open list, A* is almost the same thing but not quite
- to illustrate A* I'm going to use the same grid as before but with a different obstacle configuration
- this is one way A* performs really well--it does the minimum amount of work necessary to make maximum progress to the goal

<img src="resources/A-star.png"/>

- A* uses a so called heuristic function, which is a function that has to be set up
  - if its all zeros then A* resorts back to the search algorithm we already implemented
- if we call the heuristic function *h*, then for each cell it results into a value
- let me give you some values--its number of steps it takes to the goal if there was no obstacle
  - clearly the numbers I'm putting in right now are not reflective of the actual distance to the goal because they don't consider the obstacles
- in a world without obstacles the heuristic function that I'm giving you would actually measure the distance to the goal
- the heuristic function has to be an optimistic guess how far we are from the goal
- put differently, for any cell $(x, y)$ the heuristic function has to be an optimistic guess, which means a smaller or equal to the actual goal distance from the coordinate $(x, y)$


- that sounds a little bit ad hoc, but very often you can give good heuristic functions really easily like in this case over here
- if we just know that the agent can move left, right, up, or down, it's really easy to say what is the number of steps it would take the agent with no obstacles to get to the goal location, and that's this table over here that is easily generated automatically
- in reality this is an underestimate


- the heuristic function doesn't have to be accurate
- if it was accurate you probably already solved the planning problem
- there has to be a function that helps you understand where to search next in the case of ties, and it has to be just so that it underestimates or at best equals the true distance from the goal
- many, many problems have functions like these in our self-driving car
- it boils down much to the number of which cell steps but for the Euclidean distance to a target location


- let's work with this heuristic function from the example
- the key modification now for our search algorithm is really, really simple
- we again have an open list, and we add our state, we write down the g-value, but we also write down the g-value plus the heuristic value--this the *f-value*--this is the cumulative g-value plus the heuristic value as looked up in the table
- if I now expand, I remove the element with the lowest f-value and not the lowest g-value--that's all there is to A*
- by providing additional information, the so called heuristic function, we can guide the search
- when we have an impasse we can pick a node that looks closer to the goal state
- as a result we will likely make more progress towards the goal

# Implement A*

In [5]:
# -----------
# User Instructions:
#
# Modify the the search function so that it becomes
# an A* search algorithm as defined in the previous
# lectures.
#
# Your function should return the expanded grid
# which shows, for each element, the count when
# it was expanded or -1 if the element was never expanded.
#
# If there is no path from init to goal,
# the function should return the string 'fail'
# ----------

grid = [
    [0, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0],
]
heuristic = [
    [9, 8, 7, 6, 5, 4],
    [8, 7, 6, 5, 4, 3],
    [7, 6, 5, 4, 3, 2],
    [6, 5, 4, 3, 2, 1],
    [5, 4, 3, 2, 1, 0],
]

init = [0, 0]
goal = [len(grid) - 1, len(grid[0]) - 1]
cost = 1

delta = [[-1, 0], [0, -1], [1, 0], [0, 1]]  # go up  # go left  # go down  # go right

delta_name = ["^", "<", "v", ">"]


def search(grid, init, goal, cost, heuristic):
    # ----------------------------------------
    # modify the code below
    # ----------------------------------------
    closed = [[0 for col in range(len(grid[0]))] for row in range(len(grid))]
    closed[init[0]][init[1]] = 1

    expand = [[-1 for col in range(len(grid[0]))] for row in range(len(grid))]
    action = [[-1 for col in range(len(grid[0]))] for row in range(len(grid))]

    x = init[0]
    y = init[1]
    g = 0
    h = heuristic[x][y]
    f = g + h

    open = [[f, g, h, x, y]]

    found = False  # flag that is set when search is complete
    resign = False  # flag set if we can't find expand
    count = 0

    while not found and not resign:
        if len(open) == 0:
            resign = True
            return "Fail"
        else:
            open.sort()
            open.reverse()
            next = open.pop()
            x = next[3]
            y = next[4]
            g = next[1]
            expand[x][y] = count
            count += 1

            if x == goal[0] and y == goal[1]:
                found = True
            else:
                for i in range(len(delta)):
                    x2 = x + delta[i][0]
                    y2 = y + delta[i][1]
                    if x2 >= 0 and x2 < len(grid) and y2 >= 0 and y2 < len(grid[0]):
                        if closed[x2][y2] == 0 and grid[x2][y2] == 0:
                            g2 = g + cost
                            h2 = heuristic[x2][y2]
                            f2 = g2 + h2
                            open.append([f2, g2, h2, x2, y2])
                            closed[x2][y2] = 1

    return expand


search(grid, init, goal, cost, heuristic)

[[0, -1, -1, -1, -1, -1],
 [1, -1, -1, -1, -1, -1],
 [2, -1, -1, -1, -1, -1],
 [3, -1, 8, 9, 10, 11],
 [4, 5, 6, 7, -1, 12]]

<IPython.core.display.Javascript object>

# Dynamic Programming

- I now want to teach you an alternative method for planning
- this alternative method has a number of advantages and a number of disadvantages
- it's called dynamic programming, and just like A*, it's going to find you the shortest path
- you give it a map of the environment as in A*, one or more goal positions--let's assume just one goal position
- what it outputs is the best path from any possible starting location


- this planning technique is not just limited to a single start location, but to any start location
- why would we worry about this?--let me give you an example
- suppose you are the Google self-driving car in an environment just like this

<img src="resources/dynamic_programming.png"/>

- you're in this little street over here, and you're asked to turn right, but your goal is right over here (circle)
- as before, there are two different lanes over here--a left turn lane and a straight lane
- if you reach the straight lane, the only way to get to the goal is to go around the block over here and proceed in this direction
- the point I want to make is a different one
- that is, your attempt to do a lane shift over here might fail--it could be there's a big, big truck in this lane over here, and as you go into the right lane when you're waiting for the truck to disappear, there are these people behind you that honk their horns
- you really don't want to wait for the truck to disappear
- that means the environment is stochastic--the outcomes of actions are non-deterministic
- in our planning so far we ignored this, but in reality that's the case
- in reality, you might find yourself--wow, I'm over here (near the block)--it's happened because the world is stochastic, and this truck over here--didn't let you in
- what that means is you need a plan not just for the most likely position but you might need a plan for other positions as well


- what dynamic programming gives you is a plan for every position
- if we redraw this environment as a grid with a goal location and certain obstacles, they dynamic programming gives you an optimal action to do at every single grid cell
- as you can see, each grid cell now has a label which is often called policy, and policy is a function that maps the grid cell into an action with the action in this case as a move left, move down, move right, or move up

<img src="resources/policy.png"/>

# Computing Value

- let me make a simple example of a world like this with an obstacle over here
- say our goal state is the one in the corner over here (red)
- rather than telling you how to compute the optimal policy, which assigns an action to each of these cells, let me instead teach you about *value*
- *value function* associates to each grid cell the length of the shortest path to the goal
- for the goal, obviously, it is $0$, for each adjacent cell to the goal, it's obviously $1$, etc.
- this is recursively calculated by taking the optimal neighbor x-prime, y-prime, considering its value, and by adding the costs it takes to get there, which in our example will be plus $1$
- by applying this update equation recursively, we can attain the value function
- once we have this value function, we find that the optimal control action is obtained by minimizing the value, which is a hill-climbing type of action

<img src="resources/value_function.png"/>

# Value Program

In [6]:
# ----------
# User Instructions:
#
# Create a function compute_value which returns
# a grid of values. The value of a cell is the minimum
# number of moves required to get from the cell to the goal.
#
# If a cell is a wall or it is impossible to reach the goal from a cell,
# assign that cell a value of 99.
# ----------

grid = [
    [0, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0],
]
goal = [len(grid) - 1, len(grid[0]) - 1]
cost = 1  # the cost associated with moving from a cell to an adjacent one

delta = [[-1, 0], [0, -1], [1, 0], [0, 1]]  # go up  # go left  # go down  # go right

delta_name = ["^", "<", "v", ">"]


def compute_value(grid, goal, cost):
    # ----------------------------------------
    # insert code below
    # ----------------------------------------
    value = [[99 for row in range(len(grid[0]))] for col in range(len(grid))]

    change = True
    while change:
        change = False

        for x in range(len(grid)):
            for y in range(len(grid[0])):

                # check if goal cell
                if goal[0] == x and goal[1] == y:
                    if value[x][y] > 0:
                        value[x][y] = 0
                        change = True
                elif grid[x][y] == 0:
                    for a in range(len(delta)):
                        # project a potential next state upon executing an action
                        x2 = x + delta[a][0]
                        y2 = y + delta[a][1]

                        if (
                            x2 >= 0
                            and x2 < len(grid)
                            and y2 >= 0
                            and y2 < len(grid[0])
                            and grid[x2][y2] == 0
                        ):
                            v2 = value[x2][y2] + cost

                            if v2 < value[x][y]:
                                change = True
                                value[x][y] = v2
    return value


compute_value(grid, goal, cost)

[[11, 99, 7, 6, 5, 4],
 [10, 99, 6, 5, 4, 3],
 [9, 99, 5, 4, 3, 2],
 [8, 99, 4, 3, 2, 1],
 [7, 6, 5, 4, 99, 0]]

<IPython.core.display.Javascript object>

# Optimum Policy

In [7]:
# ----------
# User Instructions:
#
# Write a function optimum_policy that returns
# a grid which shows the optimum policy for robot
# motion. This means there should be an optimum
# direction associated with each navigable cell from
# which the goal can be reached.
#
# Unnavigable cells as well as cells from which
# the goal cannot be reached should have a string
# containing a single space (' '), as shown in the
# previous video. The goal cell should have '*'.
# ----------

grid = [
    [0, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0],
]
init = [0, 0]
goal = [len(grid) - 1, len(grid[0]) - 1]
cost = 1  # the cost associated with moving from a cell to an adjacent one

delta = [[-1, 0], [0, -1], [1, 0], [0, 1]]  # go up  # go left  # go down  # go right

delta_name = ["^", "<", "v", ">"]


def optimum_policy(grid, goal, cost):
    # ----------------------------------------
    # modify code below
    # ----------------------------------------
    value = [[99 for row in range(len(grid[0]))] for col in range(len(grid))]
    policy = [[" " for row in range(len(grid[0]))] for col in range(len(grid))]

    change = True

    while change:
        change = False

        for x in range(len(grid)):
            for y in range(len(grid[0])):
                if goal[0] == x and goal[1] == y:
                    if value[x][y] > 0:
                        policy[x][y] = "*"
                        value[x][y] = 0
                        change = True

                elif grid[x][y] == 0:
                    for a in range(len(delta)):
                        x2 = x + delta[a][0]
                        y2 = y + delta[a][1]

                        if (
                            x2 >= 0
                            and x2 < len(grid)
                            and y2 >= 0
                            and y2 < len(grid[0])
                            and grid[x2][y2] == 0
                        ):
                            v2 = value[x2][y2] + cost

                            if v2 < value[x][y]:
                                change = True
                                value[x][y] = v2
                                policy[x][y] = delta_name[a]

    return policy


optimum_policy(grid, goal, cost)

[['v', ' ', 'v', 'v', 'v', 'v'],
 ['v', ' ', 'v', 'v', 'v', 'v'],
 ['v', ' ', 'v', 'v', 'v', 'v'],
 ['v', ' ', '>', '>', '>', 'v'],
 ['>', '>', '^', '^', ' ', '*']]

<IPython.core.display.Javascript object>

# Left Turn Policy

- let's now have some fun and apply this to an actual car problem
- the one I'll using is a bit simplified as always, but it does relate to real world path planning as is done, for example, by Google Maps
- suppose we have a car down here(red)--this car now has its state defined with x, y, and an orientation, theta
- by orientation for simplicity is chosen from 4 possible directions--up, down, left, and right
- as I quiz you in the beginning, I'd like to get to the location over here, facing left
- realize that now the state space is 3-dimensional, just like in our localization example
- I now would like to implement a dynamic programming planner that gives me the optimal path for going from here to here and that let's me play with cost functions
- there are three principle actions
  - one is move in which the car just goes 1 grid cell forward in its present orientation--it doesn't turn at all
    - that could be applied anywhere in the maze in any direction
  - one is turn left and then move
  - the last one is turn right and move

In [8]:
# ----------
# User Instructions:
#
# Implement the function optimum_policy2D below.
#
# You are given a car in grid with initial state
# init. Your task is to compute and return the car's
# optimal path to the position specified in goal;
# the costs for each motion are as defined in cost.
#
# There are four motion directions: up, left, down, and right.
# Increasing the index in this array corresponds to making a
# a left turn, and decreasing the index corresponds to making a
# right turn.

forward = [[-1, 0], [0, -1], [1, 0], [0, 1]]  # go up  # go left  # go down  # go right
forward_name = ["up", "left", "down", "right"]

# action has 3 values: right turn, no turn, left turn
action = [-1, 0, 1]
action_name = ["R", "#", "L"]

# EXAMPLE INPUTS:
# grid format:
#     0 = navigable space
#     1 = unnavigable space
grid = [
    [1, 1, 1, 0, 0, 0],
    [1, 1, 1, 0, 1, 0],
    [0, 0, 0, 0, 0, 0],
    [1, 1, 1, 0, 1, 1],
    [1, 1, 1, 0, 1, 1],
]

init = [4, 3, 0]  # given in the form [row,col,direction]
# direction = 0: up
#             1: left
#             2: down
#             3: right

goal = [2, 0]  # given in the form [row,col]

cost = [2, 1, 20]  # cost has 3 values, corresponding to making
# a right turn, no turn, and a left turn

# EXAMPLE OUTPUT:
# calling optimum_policy2D with the given parameters should return
# [[' ', ' ', ' ', 'R', '#', 'R'],
#  [' ', ' ', ' ', '#', ' ', '#'],
#  ['*', '#', '#', '#', '#', 'R'],
#  [' ', ' ', ' ', '#', ' ', ' '],
#  [' ', ' ', ' ', '#', ' ', ' ']]
# ----------

# ----------------------------------------
# modify code below
# ----------------------------------------


def optimum_policy2D(grid, init, goal, cost):

    value = [
        [[999 for row in range(len(grid[0]))] for col in range(len(grid))],
        [[999 for row in range(len(grid[0]))] for col in range(len(grid))],
        [[999 for row in range(len(grid[0]))] for col in range(len(grid))],
        [[999 for row in range(len(grid[0]))] for col in range(len(grid))],
    ]

    policy = [
        [[" " for row in range(len(grid[0]))] for col in range(len(grid))],
        [[" " for row in range(len(grid[0]))] for col in range(len(grid))],
        [[" " for row in range(len(grid[0]))] for col in range(len(grid))],
        [[" " for row in range(len(grid[0]))] for col in range(len(grid))],
    ]

    policy2D = [[" " for row in range(len(grid[0]))] for col in range(len(grid))]

    change = True
    while change:
        change = False
        # go through all grid cells and calculate values

        for x in range(len(grid)):
            for y in range(len(grid[0])):
                for orientation in range(4):
                    if goal[0] == x and goal[1] == y:  # GOAL
                        if value[orientation][x][y] > 0:
                            change = True
                            value[orientation][x][y] = 0
                            policy[orientation][x][y] = "*"
                    elif grid[x][y] == 0:

                        # calculate the three ways to propagate value
                        for i in range(3):
                            o2 = (orientation + action[i]) % 4
                            x2 = x + forward[o2][0]
                            y2 = y + forward[o2][1]

                            if (
                                x2 >= 0
                                and x2 < len(grid)
                                and y2 >= 0
                                and y2 < len(grid[0])
                                and grid[x2][y2] == 0
                            ):
                                v2 = value[o2][x2][y2] + cost[i]
                                if v2 < value[orientation][x][y]:
                                    value[orientation][x][y] = v2
                                    policy[orientation][x][y] = action_name[i]
                                    change = True
    x = init[0]
    y = init[1]
    orientation = init[2]

    policy2D[x][y] = policy[orientation][x][y]
    while policy[orientation][x][y] != "*":
        if policy[orientation][x][y] == "#":
            o2 = orientation
        elif policy[orientation][x][y] == "R":
            o2 = (orientation - 1) % 4
        elif policy[orientation][x][y] == "L":
            o2 = (orientation + 1) % 4
        x = x + forward[o2][0]
        y = y + forward[o2][1]
        orientation = o2
        policy2D[x][y] = policy[orientation][x][y]

    return policy2D


optimum_policy2D(grid, init, goal, cost)

[[' ', ' ', ' ', 'R', '#', 'R'],
 [' ', ' ', ' ', '#', ' ', '#'],
 ['*', '#', '#', '#', '#', 'R'],
 [' ', ' ', ' ', '#', ' ', ' '],
 [' ', ' ', ' ', '#', ' ', ' ']]

<IPython.core.display.Javascript object>

# Planning Conclusion

- we assumed that the world is discrete, and we looked at 2 planning algorithms:
  - A*, which uses a heuristic to find a path
  - dynamic programming, which finds an entire policy that has a plan for every location